In [ ]:
import requests
from requests import Response
def get_data() ->Response | None:
    aqi_url='https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=JSON'
    try:    
        response:Response=requests.get(aqi_url)
        response.raise_for_status()
        if response.status_code==200:
            print(response)
            return response
        else:
            print('Status code error.')
            return None
    except Exception as e:
        print(f'HTTP erro occurred:{e}')
        return None
response:Response|None=get_data()
    

In [ ]:
if response:
    rawString:str=response.text

In [ ]:
from pydantic import BaseModel,Field,field_validator
import pprint
class Site(BaseModel):
    county:str
    sitename:str
    status:str
    particulate_matter:float=Field(alias='pm2.5')
    creat_time:str=Field(alias='datacreationdate')
    @field_validator('particulate_matter',mode='before')
    @classmethod
    def empty_string_pm25(cls,value):
        if value=='':
            return 0
        else:
            return value
class Sites(BaseModel):
    records:list[Site]

In [ ]:
data:Sites=Sites.model_validate_json(rawString)
download:list[dict]=data.model_dump()['records']

In [ ]:
import pandas as pd
pd.DataFrame(download)